In [ ]:
import os
import csv
from gensim.models import Word2Vec
from scipy.spatial.distance import cosine
import numpy as np
import pandas as pd
import gensim
from nltk import everygrams
import numpy as np

#### Import Word2Vec Model:

In [ ]:
model = Word2Vec.load("word2vec_model.model")
print("Model loaded successfully.")

#### Word2Vec Extract Keywords:

In [ ]:
# Find similar words 
keyword = 'timestep'
similar_words_fabric = model.wv.most_similar(keyword, topn=500)

print("Words similar to", keyword)
for word, similarity in similar_words_fabric:
    print(f"{word}: {similarity}")


In [ ]:
# Save to CSV file
csv_filename = 'similar_words_data.csv'
with open(csv_filename, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    # Write the header row
    writer.writerow(['Word', 'Similarity'])
    # Write data
    for word, similarity in similar_words_fabric:
        writer.writerow([word, similarity])

print(f"Data has been saved to {csv_filename}")

#### Load CSV:

In [ ]:
# Load the CSV file
df_raw = pd.read_csv('/Users/77y2vec/Desktop/first_paper_code/results/similar_words_output.csv')

# Create a dictionary where each row's first word is the key and the value is the list of all words in that row
words_dict_raw = df_raw.apply(lambda row: {row.iloc[0]: row.dropna().tolist()}, axis=1).to_dict()

# Combine all dictionaries into one
dict_raw = {}
for d in words_dict_raw.values():
    dict_raw.update(d)


In [ ]:
# Load the target CSV file
df_recommender = pd.read_csv('/Users/77y2vec/Desktop/first_paper_code/results/raw/similar_words_rs.csv')

# Create a dictionary from the new CSV file
words_dict_recommender = df_recommender.apply(lambda row: {row.iloc[0]: row.dropna().tolist()}, axis=1).to_dict()

# Combine all dictionaries into one
dict_recommender = {}
for d in words_dict_recommender.values():
    dict_recommender.update(d)

#### Calculate N-gram and Filter Irrelevant Words:

In [ ]:
# Function to calculate average cosine similarity between two word lists
def calculate_average_similarity(list1, list2, model):
    valid_scores = []
    for word1 in list1:
        for word2 in list2:
            if word1 in model.wv and word2 in model.wv:
                similarity_score = model.wv.similarity(word1, word2)
                valid_scores.append(similarity_score)
    return np.mean(valid_scores) if valid_scores else 0  # Return 0 if there are no valid scores

# Initialize an empty similarity matrix
similarity_matrix = np.zeros((len(dict_raw), len(dict_recommender)))

# Calculate average cosine similarity for each pair of word lists in the dictionaries
for i, (key1, words1) in enumerate(dict_raw.items()):
    for j, (key2, words2) in enumerate(dict_recommender.items()):
        similarity_matrix[i][j] = calculate_average_similarity(words1, words2, model)

# Filtering out words based on the average similarity
average_similarities = np.mean(similarity_matrix, axis=1)
filtered_keys = [key for key, avg_sim in zip(dict_raw.keys(), average_similarities) if avg_sim >= 0]

# Updating dict_algorithm to only include words with similarity >= 0
filtered_dict_raw = {key: dict_raw[key] for key in filtered_keys}

# Convert the filtered dictionary to a DataFrame and save it to a CSV file
df = pd.DataFrame.from_dict(filtered_dict_raw, orient='index').reset_index()
df.rename(columns={'index': 'Keyword'}, inplace=True)
df = df.explode('Keyword')  # If you want each value in its own row
df.to_csv('filtered_data_output.csv', index=False)

print("Filtered data has been saved!")